In [7]:
import warnings
warnings.filterwarnings("ignore")

from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import pandas as pd
pd.set_option('display.max_rows', None)
from imblearn.over_sampling import SMOTE

#(4)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_predict


In [8]:
df_crisis_boe = pd.read_excel('/content/sample_data/Bank_ready.xlsx')

df_crisis_mean = pd.read_excel('/content/sample_data/Mean_ready.xlsx') 

df_crisis_median = pd.read_excel('/content/sample_data/Median_ready.xlsx') 

df_crisis_mostf = pd.read_excel('/content/sample_data/Most_f_ready.xlsx')  

In [9]:
def datapreparation(df_crisis):

  dfx_crisis = df_crisis.drop(['year','iso'],axis=1) #retirar atributos sem relação a crises
  
  dfx_crisis = dfx_crisis.drop(['crisis'],axis=1) #retirar variavel binaria indicadora de crises

  dfy_crisis = df_crisis['crisis'].copy() 
  dfx_crisis.shape, dfy_crisis.shape
  
  dfx_crisis_resampled, dfy_crisis_resampled = SMOTE().fit_resample(dfx_crisis, dfy_crisis)
  dfx_crisis_resampled,dfy_crisis_resampled = shuffle(dfx_crisis_resampled,dfy_crisis_resampled)

  dfx_crisis_resampled = pd.DataFrame(dfx_crisis_resampled)
  dfy_crisis_resampled = pd.DataFrame(dfy_crisis_resampled)
  dfx_crisis_resampled.shape, dfy_crisis_resampled.shape

  return dfx_crisis_resampled, dfy_crisis_resampled

In [10]:
dfx_boe, dfy_boe = datapreparation(df_crisis_boe)
dfx_mean, dfy_mean = datapreparation(df_crisis_mean)
dfx_median, dfy_median = datapreparation(df_crisis_median)
dfx_mostf, dfy_mostf = datapreparation(df_crisis_mostf)

In [11]:
dfx_boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2308 non-null   float64
 1   1       2308 non-null   float64
 2   2       2308 non-null   float64
 3   3       2308 non-null   float64
 4   4       2308 non-null   float64
 5   5       2308 non-null   float64
 6   6       2308 non-null   float64
 7   7       2308 non-null   float64
 8   8       2308 non-null   float64
 9   9       2308 non-null   float64
 10  10      2308 non-null   float64
 11  11      2308 non-null   float64
dtypes: float64(12)
memory usage: 216.5 KB


In [12]:
dfy_boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       2308 non-null   int64
dtypes: int64(1)
memory usage: 18.2 KB


In [13]:
# Split the dataset in two equal parts
def spliter(dfx,dfy):
  X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.75, random_state=0)
  return X_train, X_test, y_train, y_test

In [14]:
# Set the parameters by cross-validation
tuned_parameters = [{'penalty': ['elasticnet','l1','none','l2'],
                     'tol': [1,1e-1,1e-3,1e-5,1e-6],
                     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
                     'multi_class': ['auto','ovr','multinomial'],
                     'C': [0.001,0.01,0.1,1],
                     'max_iter': [50,100,300,500,1000]},
                    ]

scores = ['precision', 'recall']


In [15]:
def p_estimation(dfx,dfy):
  X_train, X_test, y_train, y_test = spliter(dfx,dfy)
  for score in scores:
      print("# Tuning hyper-parameters for %s" % score)
      print()

      LogR = GridSearchCV(
          LogisticRegression(), tuned_parameters, scoring='%s_macro' % score
      )
      LogR.fit(X_train, y_train)

      print("Best parameters set found on development set:")
      print()
      print(LogR.best_params_)
      print()
      print("Grid scores on development set:")
      print()
      means = LogR.cv_results_['mean_test_score']
      stds = LogR.cv_results_['std_test_score']
      for mean, std, params in zip(means, stds, LogR.cv_results_['params']):
          print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
      print()

      print("Detailed classification report:")
      print()
      print("The model is trained on the full development set.")
      print("The scores are computed on the full evaluation set.")
      print()
      y_true, y_pred = y_test, LogR.predict(X_test)
      print(classification_report(y_true, y_pred))
      print()

In [ ]:
p_estimation(dfx_boe, dfy_boe)

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 0.001, 'max_iter': 100, 'multi_class': 'auto', 'penalty': 'none', 'solver': 'newton-cg', 'tol': 0.001}

Grid scores on development set:

nan (+/-nan) for {'C': 0.001, 'max_iter': 50, 'multi_class': 'auto', 'penalty': 'elasticnet', 'solver': 'newton-cg', 'tol': 1}
nan (+/-nan) for {'C': 0.001, 'max_iter': 50, 'multi_class': 'auto', 'penalty': 'elasticnet', 'solver': 'newton-cg', 'tol': 0.1}
nan (+/-nan) for {'C': 0.001, 'max_iter': 50, 'multi_class': 'auto', 'penalty': 'elasticnet', 'solver': 'newton-cg', 'tol': 0.001}
nan (+/-nan) for {'C': 0.001, 'max_iter': 50, 'multi_class': 'auto', 'penalty': 'elasticnet', 'solver': 'newton-cg', 'tol': 1e-05}
nan (+/-nan) for {'C': 0.001, 'max_iter': 50, 'multi_class': 'auto', 'penalty': 'elasticnet', 'solver': 'newton-cg', 'tol': 1e-06}
nan (+/-nan) for {'C': 0.001, 'max_iter': 50, 'multi_class': 'auto', 'penalty': 'elasticnet', 'solver': 'lbfgs', 'tol': 

In [ ]:
p_estimation(dfx_mean, dfy_mean)

In [ ]:
p_estimation(dfx_median, dfy_median)

In [ ]:
p_estimation(dfx_mostf, dfy_mostf)